In [1]:
from google.colab import drive
import numpy as np
import cv2, os, torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score

# Read the images (function from the first seminar)

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
def read_files(X, Y, path, ans):
  files = os.listdir(path)
  for name in files:
    img = cv2.imread(path + '/' + name, 0)
    if img.shape != 0:
      img = cv2.resize(img, (256, 256))
      vect = img.reshape(1, 256 ** 2)
      vect = vect / 255.
      X = vect if (X is None) else np.vstack((X, vect)) 
      Y = np.append(Y, ans)
  return X, Y

In [4]:
path = "/content/drive/MyDrive/lesson1_dataset"
X = None
Y = np.array([])
X, Y = read_files(X, Y, path + "/logloss_0", 0)
X, Y = read_files(X, Y, path + "/logloss_1", 1)

# Create the dataset


In [5]:
class CreateDataset(Dataset):
    def __init__(self, X, Y):
        self.x = torch.from_numpy(X)
        self.y = torch.from_numpy(Y)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]
      
    def __len__(self):
        return self.x.shape[0]

In [6]:
np.random.seed(1)
dataset = CreateDataset(X, Y)
train, test = torch.utils.data.random_split(dataset, [round(0.8*len(dataset)), round(0.2*len(dataset))])

In [7]:
train_loader = DataLoader(dataset=train, batch_size=16, shuffle=True)
test_loader = DataLoader(dataset=test, shuffle=True)

In [8]:
for x,y in train_loader:
  print(x.view(x.shape[0], -1).shape, y.shape)
  break

torch.Size([16, 65536]) torch.Size([16])


# Configure the model

In [9]:
model = nn.Sequential(
    nn.Linear(65536, 2048),
    nn.BatchNorm1d(2048),
    nn.Dropout(0.5),
    nn.ReLU(),
    nn.Linear(2048, 2048),
    nn.BatchNorm1d(2048),
    nn.ReLU(),
    nn.Linear(2048, 2048),
    nn.BatchNorm1d(2048),
    nn.ReLU(),
    nn.Linear(2048,2048),
    nn.BatchNorm1d(2048),
    nn.Dropout(0.5),
    nn.ReLU(),
    nn.Linear(2048, 2),
    nn.Softmax()
)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [11]:
model.cuda()

Sequential(
  (0): Linear(in_features=65536, out_features=2048, bias=True)
  (1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): ReLU()
  (4): Linear(in_features=2048, out_features=2048, bias=True)
  (5): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU()
  (7): Linear(in_features=2048, out_features=2048, bias=True)
  (8): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (9): ReLU()
  (10): Linear(in_features=2048, out_features=2048, bias=True)
  (11): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): Dropout(p=0.5, inplace=False)
  (13): ReLU()
  (14): Linear(in_features=2048, out_features=2, bias=True)
  (15): Softmax(dim=None)
)

# Training

In [12]:
epochs = 20
model.train()
for i in range(epochs):
    for j, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()
        x = x.view(x.shape[0], -1)
        x = x.cuda()
        y = y.cuda()
        y_pred = model(x.float())
        loss = criterion(y_pred, y.long())
        print(f"Epoch {i}\t iter {j}\t loss {loss}")
        loss.backward()
        optimizer.step()

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch 0	 iter 0	 loss 0.6572994589805603
Epoch 0	 iter 1	 loss 0.5254168510437012
Epoch 0	 iter 2	 loss 0.3589804768562317
Epoch 0	 iter 3	 loss 0.5217317938804626
Epoch 1	 iter 0	 loss 0.5469618439674377
Epoch 1	 iter 1	 loss 0.3879917562007904
Epoch 1	 iter 2	 loss 0.4894407391548157
Epoch 1	 iter 3	 loss 0.31327298283576965
Epoch 2	 iter 0	 loss 0.49811315536499023
Epoch 2	 iter 1	 loss 0.3730299472808838
Epoch 2	 iter 2	 loss 0.37833499908447266
Epoch 2	 iter 3	 loss 0.5633803606033325
Epoch 3	 iter 0	 loss 0.4455563426017761
Epoch 3	 iter 1	 loss 0.47449222207069397
Epoch 3	 iter 2	 loss 0.4343473017215729
Epoch 3	 iter 3	 loss 0.5711500644683838
Epoch 4	 iter 0	 loss 0.3688523471355438
Epoch 4	 iter 1	 loss 0.37728050351142883
Epoch 4	 iter 2	 loss 0.46270787715911865
Epoch 4	 iter 3	 loss 0.5044106245040894
Epoch 5	 iter 0	 loss 0.4518081545829773
Epoch 5	 iter 1	 loss 0.3159264922142029
Epoch 5	 iter 2	 loss 0.3764341175556183
Epoch 5	 iter 3	 loss 0.7146843671798706
Epoch 6	 i

# Evaluation of the model

In [13]:
model.eval()
y_true = []; y_pred = []
for x, y in test_loader:
  x = x.cuda()
  y = y.cuda()
  y_pred.append(round(model(x.float()).data[0][1].item()))
  y_true.append(int(y))

print('The accuracy of the model is:', accuracy_score(y_true, y_pred))

The accuracy of the model is: 0.9230769230769231


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
